In [1]:
from google.colab import files

uploaded = files.upload()


Saving persian_english_french_3000_v3.csv to persian_english_french_3000_v3.csv


In [2]:
import pandas as pd

df = pd.read_csv("persian_english_french_3000_v3.csv")

df.head()


,persian,english,french
0,ما روز خوبی را با هم گذراندیم.,We had a good day together.,Nous avons passé une bonne journée ensemble.
1,ما درباره این موضوع بحث کردیم.,We discussed this topic.,Nous avons discuté de ce sujet.
2,من به تصمیم تو احترام می‌گذارم.,I respect your decision.,Je respecte ta décision.
3,من نظر تو را درک می‌کنم.,I understand your point of view.,Je comprends ton point de vue.
4,این تصمیم آسانی نبود. و این تجربه را فراموش نم...,This decision was not easy. and I will not for...,Cette décision n’était pas facile. et je n’oub...


In [3]:
# Randomly sample 1,000 examples for evaluation only
eval_df = df.sample(n=1000, random_state=42).reset_index(drop=True)

eval_df.shape


(1000, 3)

In [4]:
eval_df.head(2)


,persian,english,french
0,من حقیقت را از او پنهان نکردم. و احساس خوبی دا...,I did not hide the truth from him. and I felt ...,Je ne lui ai pas caché la vérité. et je me sen...
1,ما در کافه‌ای کوچک قهوه نوشیدیم.,We drank coffee in a small café.,Nous avons bu du café dans un petit café.


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()

print("Model and tokenizer loaded successfully!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Model and tokenizer loaded successfully!


In [6]:
def translate(text, src_lang, tgt_lang, max_length=128):
    tokenizer.src_lang = src_lang
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
            max_length=max_length
        )

    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Test sentence
test_sentence = eval_df.loc[0, "persian"]

print("Persian:", test_sentence)
print("French:", translate(test_sentence, "fas_Arab", "fra_Latn"))


Persian: من حقیقت را از او پنهان نکردم. و احساس خوبی داشتم.
French: Je ne lui ai pas caché la vérité, et j'ai eu une bonne impression.


In [7]:
from tqdm import tqdm

def batch_translate(texts, src_lang, tgt_lang, batch_size=8, max_length=128):
    translations = []
    tokenizer.src_lang = src_lang

    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(model.device)

        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
                max_length=max_length
            )

        decoded = tokenizer.batch_decode(
            generated_tokens, skip_special_tokens=True
        )
        translations.extend(decoded)

    return translations


# Run direct translation: Persian -> French
direct_fr = batch_translate(
    eval_df["persian"].tolist(),
    src_lang="fas_Arab",
    tgt_lang="fra_Latn"
)

eval_df["direct_fa_fr"] = direct_fr

eval_df.head(2)


100%|██████████| 125/125 [00:47<00:00,  2.61it/s]


,persian,english,french,direct_fa_fr
0,من حقیقت را از او پنهان نکردم. و احساس خوبی دا...,I did not hide the truth from him. and I felt ...,Je ne lui ai pas caché la vérité. et je me sen...,"Je ne lui ai pas caché la vérité, et j'ai eu u..."
1,ما در کافه‌ای کوچک قهوه نوشیدیم.,We drank coffee in a small café.,Nous avons bu du café dans un petit café.,Nous avons bu un petit café au café.


In [8]:
from tqdm import tqdm

def batch_translate(texts, src_lang, tgt_lang, batch_size=8, max_length=128):
    translations = []
    tokenizer.src_lang = src_lang

    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(model.device)

        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
                max_length=max_length
            )

        decoded = tokenizer.batch_decode(
            generated_tokens, skip_special_tokens=True
        )
        translations.extend(decoded)

    return translations


# Run direct translation: Persian -> French
direct_fr = batch_translate(
    eval_df["persian"].tolist(),
    src_lang="fas_Arab",
    tgt_lang="fra_Latn"
)

eval_df["direct_fa_fr"] = direct_fr

eval_df.head(2)


100%|██████████| 125/125 [00:47<00:00,  2.62it/s]


,persian,english,french,direct_fa_fr
0,من حقیقت را از او پنهان نکردم. و احساس خوبی دا...,I did not hide the truth from him. and I felt ...,Je ne lui ai pas caché la vérité. et je me sen...,"Je ne lui ai pas caché la vérité, et j'ai eu u..."
1,ما در کافه‌ای کوچک قهوه نوشیدیم.,We drank coffee in a small café.,Nous avons bu du café dans un petit café.,Nous avons bu un petit café au café.


In [9]:
# Persian -> English
pivot_en = batch_translate(
    eval_df["persian"].tolist(),
    src_lang="fas_Arab",
    tgt_lang="eng_Latn"
)

eval_df["pivot_en"] = pivot_en

eval_df.head(2)


100%|██████████| 125/125 [00:37<00:00,  3.32it/s]


,persian,english,french,direct_fa_fr,pivot_en
0,من حقیقت را از او پنهان نکردم. و احساس خوبی دا...,I did not hide the truth from him. and I felt ...,Je ne lui ai pas caché la vérité. et je me sen...,"Je ne lui ai pas caché la vérité, et j'ai eu u...","I didn't hide the truth from him, and I felt g..."
1,ما در کافه‌ای کوچک قهوه نوشیدیم.,We drank coffee in a small café.,Nous avons bu du café dans un petit café.,Nous avons bu un petit café au café.,We had a little coffee in the cafe.


In [10]:
# English -> French
pivot_fr = batch_translate(
    eval_df["pivot_en"].tolist(),
    src_lang="eng_Latn",
    tgt_lang="fra_Latn"
)

eval_df["pivot_fa_en_fr"] = pivot_fr

eval_df.head(2)


100%|██████████| 125/125 [00:51<00:00,  2.42it/s]


,persian,english,french,direct_fa_fr,pivot_en,pivot_fa_en_fr
0,من حقیقت را از او پنهان نکردم. و احساس خوبی دا...,I did not hide the truth from him. and I felt ...,Je ne lui ai pas caché la vérité. et je me sen...,"Je ne lui ai pas caché la vérité, et j'ai eu u...","I didn't hide the truth from him, and I felt g...","Je ne lui avais pas caché la vérité, et je me ..."
1,ما در کافه‌ای کوچک قهوه نوشیدیم.,We drank coffee in a small café.,Nous avons bu du café dans un petit café.,Nous avons bu un petit café au café.,We had a little coffee in the cafe.,On a bu un café au café.


In [13]:
from sacrebleu import corpus_bleu

references = [eval_df["french"].tolist()]  # sacreBLEU expects list of references
direct_outputs = eval_df["direct_fa_fr"].tolist()
pivot_outputs = eval_df["pivot_fa_en_fr"].tolist()


In [14]:
direct_bleu = corpus_bleu(direct_outputs, references)
pivot_bleu = corpus_bleu(pivot_outputs, references)

print("Direct FA → FR BLEU:", direct_bleu.score)
print("Pivot FA → EN → FR BLEU:", pivot_bleu.score)


Direct FA → FR BLEU: 38.0296022761084
Pivot FA → EN → FR BLEU: 40.032669504570755


In [15]:
import pandas as pd

results_df = pd.DataFrame({
    "Setup": ["Direct (fa→fr)", "Pivot (fa→en→fr)"],
    "BLEU": [direct_bleu.score, pivot_bleu.score]
})

results_df


,Setup,BLEU
0,Direct (fa→fr),38.029602
1,Pivot (fa→en→fr),40.032670


In [16]:
from sacrebleu import corpus_chrf

direct_chrf = corpus_chrf(direct_outputs, references)
pivot_chrf = corpus_chrf(pivot_outputs, references)

print("Direct FA → FR chrF++:", direct_chrf.score)
print("Pivot FA → EN → FR chrF++:", pivot_chrf.score)


Direct FA → FR chrF++: 65.65008400917752
Pivot FA → EN → FR chrF++: 66.96771644220256


In [17]:
!pip install -q bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00


In [18]:
from bert_score import score

refs = eval_df["french"].tolist()
direct_hyps = eval_df["direct_fa_fr"].tolist()
pivot_hyps = eval_df["pivot_fa_en_fr"].tolist()

P_d, R_d, F1_d = score(
    direct_hyps,
    refs,
    lang="fr",
    model_type="xlm-roberta-large",
    verbose=True
)

P_p, R_p, F1_p = score(
    pivot_hyps,
    refs,
    lang="fr",
    model_type="xlm-roberta-large",
    verbose=True
)

print("Direct FA → FR BERTScore F1:", F1_d.mean().item())
print("Pivot FA → EN → FR BERTScore F1:", F1_p.mean().item())


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

done in 1.99 seconds, 503.74 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 1.73 seconds, 576.84 sentences/sec
Direct FA → FR BERTScore F1: 0.9624955058097839
Pivot FA → EN → FR BERTScore F1: 0.9637450575828552


In [19]:
import pandas as pd

results_df = pd.DataFrame({
    "Setup": [
        "Direct (FA → FR)",
        "Pivot (FA → EN → FR)"
    ],
    "BLEU ↑": [
        38.0296,
        40.0327
    ],
    "chrF++ ↑": [
        65.6501,
        66.9677
    ],
    "BERTScore F1 ↑": [
        0.9625,
        0.9637
    ]
})

results_df


,Setup,BLEU ↑,chrF++ ↑,BERTScore F1 ↑
0,Direct (FA → FR),38.0296,65.6501,0.9625
1,Pivot (FA → EN → FR),40.0327,66.9677,0.9637


In [12]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.7 MB/s eta 0:00:00
